# Лекция 3

### Импортирование Модулей


### Основные рабочие конструкции в Haskell

- Сопоставление шаблонов
- Гварды
- Where
- Let .. in 
- Case
- Рекурсия 
- Функции высшего порядка
    - lambda функции
    - map
    - filter
    - foldr/foldl
    - композиция функций
    
### Типы и Классы типов
- data 
- newtype
- type


### Импортирование Модулей

In [42]:
-- Импортирование Модулей
import Data.List  -- простой импорт, в области видимости появляются все экспортируемые модулем элементы

import qualified Data.String as S -- импорт с именованием, к элементам в модуле можно теперь обращаться через . 
                                  -- , например S.words - обращение к функции words в модуле Data.String
                                  
import Data.Text ( unpack  -- импорт из модуля только указанных функций
                 , pack)
                 
import Data.Char hiding (isNumber) -- импортирование всех элементов из модуля. кроме перечисленных в скобках
:set -XRankNTypes

### Основные рабочие конструкции в Haskell
Так как в Haskell нет циклов и переменных, для создания функций приходится использовать альтернативные конструкции


#### Сопоставление шаблонов
При определении функций можно определить разные тела функций для различных примеров входов

In [43]:
sayNumber :: forall a.Integral a => a -> String 
sayNumber 1 = "One!"       -- при вычислении значния функции сработает первое 
sayNumber 2 = "Two!"       -- тело функции, для которого совпадёт патерн входного значния 
sayNumber 3 = "Three!"     -- таким образом, 
sayNumber x = "A lot of!"  -- наиболее общий шаблон должен быть в конце.

In [44]:
sayNumber 3
sayNumber 51

"Three!"

"A lot of!"

In [45]:
-- data Maybe a = Nothing | Just a --тип-обёртка, имеющий смысл вычислени, которые могут "сломаться". 
maybeHead :: [a] -> Maybe a
maybeHead [] = Nothing             -- если  список пустой, то и первого элемента нет, поэтому возвращаем Nothing
maybeHead (x:_) = Just x          -- в противном случае, возвращаяем первый элемент списка

In [ ]:
maybeHead []
maybeHead [1,2,3]

In [ ]:
first :: (a,b) -> a  -- так же сопоставление шаблонов может разбирать кортэжи
first (x,_) = x      --  _ обозначается часть шаблона, которая для нас не важна, то что мы собрались откидывать

second :: (a,b) -> b
second (_,y) = y

swap :: (a,b) -> (b,a)  -- реализация функции, меняющей местами элементы пары, выглядит особенно изящно
swap (x,y) = (y,x)

In [ ]:
first ('a',1)
second ('a',1)
swap ('a',1)

In [1]:
firstLetter :: String -> String
firstLetter "" = "Empty string, whoops!" 
firstLetter all@(x:xs) = "The first letter of " ++ all ++ " is " ++ [x] 

In [2]:
firstLetter "Hello!"

"The first letter of Hello! is H"

#### Гварды
Чем-то похожи на множественный if.
Синтаксически, каждое новое условие пишется после знака |. Условие должно возвращать булево значение, сработает первый из гвардов, в котором условие даст True. Последний гвард пишется без условия, вместо него otherwise, туда сваливаются те значения, проверка которых во всех других гвардах дала False.   

In [4]:
max' :: (Ord a) => a -> a -> a  
max' a b   
    | a > b     = a  
    | otherwise = b  
    
max'' :: (Ord a) => a -> a -> a  
max'' a b | a > b = a | otherwise = b -- так же гварды можно записывать в строку, но это менее читаемо

#### Where 
На примере функции расчёта индекса массы тела, расссмотрим как можно использовать гварды и Where

In [9]:
bmiTell0 :: (RealFloat a) => a -> String  -- самый простой вариант испоьзования гвардов
bmiTell0 bmi  
    | bmi <= 18.5 = "You're underweight, you emo, you!"  
    | bmi <= 25.0 = "You're supposedly normal. Pffft, I bet you're ugly!"  
    | bmi <= 30.0 = "You're fat! Lose some weight, fatty!"  
    | otherwise   = "You're a whale, congratulations!"

bmiTell1 :: (RealFloat a) => a -> a -> String  -- в условиях происходят вычисления
bmiTell1 weight height  
    | weight / height ^ 2 <= 18.5 = "You're underweight, you emo, you!"  
    | weight / height ^ 2 <= 25.0 = "You're supposedly normal. Pffft, I bet you're ugly!"  
    | weight / height ^ 2 <= 30.0 = "You're fat! Lose some weight, fatty!"  
    | otherwise                 = "You're a whale, congratulations!"  
    
bmiTell2 :: (RealFloat a) => a -> a -> String  -- вычисления переобозначины в конструкции Where
bmiTell2 weight height  
    | bmi <= 18.5 = "You're underweight, you emo, you!"  
    | bmi <= 25.0 = "You're supposedly normal. Pffft, I bet you're ugly!"  
    | bmi <= 30.0 = "You're fat! Lose some weight, fatty!"  
    | otherwise   = "You're a whale, congratulations!"  
    where bmi = weight / height ^ 2  
    
bmiTell3 :: (RealFloat a) => a -> a -> String  -- кроме того, в конструкции Where так же 
bmiTell3 weight height                         -- переобозначены и пороговые значения
    | bmi <= skinny = "You're underweight, you emo, you!"  
    | bmi <= normal = "You're supposedly normal. Pffft, I bet you're ugly!"  
    | bmi <= fat    = "You're fat! Lose some weight, fatty!"  
    | otherwise     = "You're a whale, congratulations!"  
    where bmi = weight / height ^ 2  
          skinny = 18.5  
          normal = 25.0  
          fat = 30.0 
          
bmiTell4 :: (RealFloat a) => a -> a -> String  
bmiTell4 weight height  
    | bmi <= skinny = "You're underweight, you emo, you!"  
    | bmi <= normal = "You're supposedly normal. Pffft, I bet you're ugly!"  
    | bmi <= fat    = "You're fat! Lose some weight, fatty!"  
    | otherwise     = "You're a whale, congratulations!"            
    where bmi = weight / height ^ 2  
          (skinny, normal, fat) = (18.5, 25.0, 30.0)  -- пороговые значения переобозначены с помощью 
                                                      -- синтаксиса сопоставления шаблонов

In [6]:
--ещё один хороший пример сопоставления шаблонов и конструкции Where
initials :: String -> String -> String  --  функция возвращает инициалы по имени и фамилии 
initials firstname lastname = [f] ++ ". " ++ [l] ++ "."  
    where (f:_) = firstname  -- вообще говоря, тут синтаксис шаблонов не перекрывает все случаи. 
          (l:_) = lastname   -- При вызве функции с пустой строкой, возникнет ошибка

In [7]:
initials "Semion" "Shishkin"

"S. S."

In [8]:
    initials "Semion" ""

: 

#### Let .. in ..
похожа на наконструкцию Where, но сначала идёт переобозначение выраения. а потом его использование.
Так же let .. in .. свмо является выражением, тогда как Where синтаксическая конструкция.

In [10]:
cylinder :: (RealFloat a) => a -> a -> a  -- функция, сассчитывающая площадь цилиндра 
cylinder r h = 
    let sideArea = 2 * pi * r * h  
        topArea = pi * r ^2  
    in  sideArea + 2 * topArea  

In [11]:
cylinder 3 10

245.04422698000386

In [13]:
[if 5 > 3 then "Woo" else "Boo", if 'a' > 'b' then "Foo" else "Bar"] -- if - выражение

4 * (let a = 9 in a + 1) + 2  -- так же как и let .. in .. 

[let square x = x * x in (square 5, square 3, square 2)] -- даже так!

(let a = 100; b = 200; c = 300 in a*b*c, let foo="Hey "; bar = "there!" in foo ++ bar) 



["Woo","Bar"]

42

[(25,9,4)]

(6000000,"Hey there!")

#### Case
 в общем виде, выглядит так :
 
 _**case**_ expression _**of**_ pattern -> result  
                                pattern -> result  
                                pattern -> result  
  ...
                   
case так же является выражением!
вообще говоря, сопоставление шаблонов является синтаксический сахар для case конструкции 

In [14]:
describeList :: [a] -> String  
describeList xs = "The list is " ++ case xs of [] -> "empty."  
                                               [x] -> "a singleton list."   
                                               xs -> "a longer list." 
                                        
                                        
describeList' :: [a] -> String  -- то же самое, но с Where 
describeList' xs = "The list is " ++ what xs  
    where what [] = "empty."  
          what [x] = "a singleton list."  
          what xs = "a longer list."  

#### Рекурсия
В программировании рекурсия — вызов функции (процедуры) из неё же самой, непосредственно (простая рекурсия) или через другие функции (сложная или косвенная рекурсия), например, функция **A**￼ вызывает функцию **B**, а функция **B**￼ — функцию **A**. Количество вложенных вызовов функции или процедуры называется глубиной рекурсии. Рекурсивная программа позволяет описать повторяющееся или даже потенциально бесконечное вычисление, причём без явных повторений частей программы и использования циклов.

Чтобы рекурсия не была бесконечной, нужно сделать условие выхода из рекурсии!

In [15]:
factorial :: (Integral a) => a -> a  -- рекурсивная функция, считающая факториял числа
factorial 0 = 1                      -- граничное условие выхода из рекурсии
factorial n = n * factorial (n - 1)  -- рекурсивная запись. Это так называемая, хвостовая рекурсия

length' :: (Num b) => [a] -> b       -- длина списка
length' [] = 0                       -- условие выхода
length' (_:xs) = 1 + length' xs      -- Хвостовая рекурсия преобразуется в цикл

sum' :: (Num a) => [a] -> a          -- сумма спика чисел
sum' [] = 0                          -- условие выхода
sum' (x:xs) = x + sum' xs            -- рекурсивных выход

In [16]:
maximum' :: (Ord a) => [a] -> a  -- максимум в списке 
maximum' [] = error "maximum of empty list"  
maximum' [x] = x  
maximum' (x:xs)   
    | x > maxTail = x  
    | otherwise = maxTail  
    where maxTail = maximum' xs 

In [18]:
replicate' :: (Num i, Ord i) => i -> a -> [a]  -- эта функция возвращает сисок,
replicate' n x  
    | n <= 0    = []  
    | otherwise = x:replicate' (n-1) x  

In [20]:
replicate' 4 "a"

["a","a","a","a"]

In [21]:
repeat' :: a -> [a]       -- эта функция возвращает бесконечный список повторений элемента х
repeat' x = x:repeat' x   -- вообще говоря, это корекурсия

In [22]:
elem' :: (Eq a) => a -> [a] -> Bool  -- функция, говорящая, является ли входное значение элементом списка?
elem' a [] = False  
elem' a (x:xs)  
    | a == x    = True  
    | otherwise = a `elem'` xs 

In [27]:
quicksort :: (Ord a) => [a] -> [a]  
quicksort [] = []  
quicksort (x:xs) =   
    let smallerSorted = quicksort [a | a <- xs, a <= x]  
        biggerSorted = quicksort [a | a <- xs, a > x]  
    in   smallerSorted ++ [x] ++ biggerSorted 

In [28]:
quicksort [10,2,5,3,1,6,7,4,2,3,4,8,9]


[1,2,2,3,3,4,4,5,6,7,8,9,10]

#### Функции высшего порядка 
Вообще говоря, это функции, кторые принимают в качества аргумента функции и  взвращают функции


In [ ]:
applyTwice :: (a -> a) -> a -> a  -- например, функция, которая на вход принимает функцию и значение
applyTwice f x = f (f x)          -- и возвращает результат приминения функциик знаению два раза

applyTwice (++ " HAHA") "HEY"  
"HEY HAHA HAHA"

flip' :: (a -> b -> c) -> b -> a -> c  -- меняет агрументы функции двух аргументов местами
flip' f y x = f x y  

##### лямбда-функции 
Это безымянные,"быстрые" функции
\ args -> val

In [29]:
flip' :: (a -> b -> c) -> b -> a -> c  --тот же пример, но с лямбда-функцией
flip' f = \x y -> f y x 

##### map 
map :: (a -> b) -> [a] -> [b]  
map _ [] = []  
map f (x:xs) = f x : map f xs  

применяет функцию ко всем элементам списка и возвращает новый список с результатом


In [30]:
map (+3) [1,5,3,1,6] 
map (++ "!") ["BIFF", "BANG", "POW"] 
map (replicate 3) [3..6]  
map (*) [0..]

[4,8,6,4,9]

["BIFF!","BANG!","POW!"]

[[3,3,3],[4,4,4],[5,5,5],[6,6,6]]

##### filter

filter :: (a -> Bool) -> [a] -> [a]  
filter _ [] = []  
filter p (x:xs)   
    | p x       = x : filter p xs  
    | otherwise = filter p xs  

пробегает по списку и возвращает толко теэлементы, в которых предикат истинный


In [33]:
filter (>3) [1,5,3,2,1,6,4,3,2,1] 
filter (==3) [1,2,3,4,5] 
filter (`elem` ['a'..'z']) "u LaUgH aT mE BeCaUsE I aM diFfeRent"  
filter (`elem` ['A'..'Z']) "i lauGh At You BecAuse u r aLL the Same"  

[5,6,4]

[3]

"uagameasadifeent"

"GAYBALLS"

In [34]:
quicksort :: (Ord a) => [a] -> [a]    -- реаизация быстрой сортировки с функцией filter
quicksort [] = []    
quicksort (x:xs) =     
    let smallerSorted = quicksort (filter (<=x) xs)  
        biggerSorted = quicksort (filter (>x) xs)   
    in  smallerSorted ++ [x] ++ biggerSorted

#### foldl
складывает (или сворачивает) список к одному значению, слева  направо

In [36]:
:t foldl
:t foldr

foldl :: forall (t :: * -> *) b a. Foldable t => (b -> a -> b) -> b -> t a -> b

foldr :: forall (t :: * -> *) a b. Foldable t => (a -> b -> b) -> b -> t a -> b

In [37]:
sum' :: (Num a) => [a] -> a               -- на примере функции, считающей сумму элементов в списке
sum' xs = foldl (\acc x -> acc + x) 0 xs  -- рассмотрим действие foldl.
 -- в качестве аргументов, она берёт функцию двух аргументов(аккумулятор и непосредственно значение из списка),
 -- аккумулятор, и список элементов, и возвращает одно значение

In [38]:
sum' [3,5,2,1]  

11

In [39]:
sum' :: (Num a) => [a] -> a  --можно написать и так, опустив параметры 
sum' = foldl (+) 0 

In [40]:
elem' :: (Eq a) => a -> [a] -> Bool  -- а как вам такой вариант функции, ищущей элемент в списке?
elem' y ys = foldl (\acc x -> if x == y then True else acc) False ys 

#### foldr
почти то же, что и foldl, 
складывает (или сворачивает) список к одному значению, но  справа  налево

In [41]:
map' :: (a -> b) -> [a] -> [b]                -- например, реализация map через foldr
map' f xs = foldr (\x acc -> f x : acc) [] xs 

-- map' f xs = foldl (\acc x -> acc ++ [f x]) [] xs, 
--- хотя можно было бы сделать и так, но ++ намного дороже, чем :